# Method 1: Enhanced Augmentation - FER2013 Training

In [ ]:
!nvidia-smi

import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print(f"TensorFlow: {tf.__version__}")
print(f"GPUs: {gpus}")

if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("✅ GPU ENABLED!")
else:
    print("❌ NO GPU! Go to Runtime → Change runtime type → GPU")


In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import warnings
import pickle
import json
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

print("✅ Libraries imported!")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("✅ Drive mounted!")


In [ ]:
import os

DATASET_PATH = '/content/drive/MyDrive/CaptoneProject/camera'
ZIP_PATH = '/content/drive/MyDrive/CaptoneProject/camera.zip'

if os.path.exists(ZIP_PATH):
    print(f"✅ ZIP found at {ZIP_PATH}")
else:
    print("⚠️ ZIP not found! Please run the specific zip creation cell if needed.")


In [ ]:
import os
import shutil

ZIP_PATH = '/content/drive/MyDrive/CaptoneProject/camera.zip'
LOCAL_PATH = '/content/dataset'

if not os.path.exists(LOCAL_PATH):
    if os.path.exists(ZIP_PATH):
        print("📦 Unzipping... (this might take a moment)")
        !unzip -q -o "{ZIP_PATH}" -d /content/
        
        # Handle directory structure
        if os.path.exists('/content/camera'):
            !mv /content/camera "{LOCAL_PATH}"
        elif os.path.exists('/content/train') and os.path.exists('/content/test'):
            os.makedirs(LOCAL_PATH, exist_ok=True)
            !mv /content/train "{LOCAL_PATH}/train"
            !mv /content/test "{LOCAL_PATH}/test"
        print("✅ Dataset ready at /content/dataset")
    else:
        print("❌ ZIP file not found in Drive!")
else:
    print("✅ Dataset already exists locally!")

TRAIN_DIR = os.path.join(LOCAL_PATH, 'train')
TEST_DIR = os.path.join(LOCAL_PATH, 'test')


In [ ]:
# ========================================
# CONFIG: METHOD 1 - ENHANCED AUGMENTATION
# ========================================
IMG_SIZE = 48
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 0.0005
NUM_CLASSES = 7
EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
SEED = 42
np.random.seed(SEED); tf.random.set_seed(SEED)

CHECKPOINT_DIR = '/content/drive/MyDrive/CaptoneProject/checkpoints/method1_aug'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

MODEL_CHECKPOINT_PATH = f'{CHECKPOINT_DIR}/model_checkpoint.keras'
BEST_MODEL_PATH = f'{CHECKPOINT_DIR}/best_model.keras'
HISTORY_PATH = f'{CHECKPOINT_DIR}/training_history.pkl'
print(f"📁 Checkpoints Saving to: {CHECKPOINT_DIR}")


In [ ]:
# Enhanced Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest',
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=(IMG_SIZE, IMG_SIZE), color_mode='grayscale', batch_size=BATCH_SIZE, class_mode='categorical', subset='training', shuffle=True, seed=SEED)
validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=(IMG_SIZE, IMG_SIZE), color_mode='grayscale', batch_size=BATCH_SIZE, class_mode='categorical', subset='validation', shuffle=False, seed=SEED)
test_generator = test_datagen.flow_from_directory(
    TEST_DIR, target_size=(IMG_SIZE, IMG_SIZE), color_mode='grayscale', batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)


In [ ]:
train_labels = train_generator.classes
class_weights_array = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = dict(enumerate(class_weights_array))

print("📊 Class Weights calculated!")


In [ ]:
# Standard Custom CNN (Same as original)
def build_cnn(input_shape=(48, 48, 1), num_classes=7):
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), padding='same', input_shape=input_shape), layers.BatchNormalization(), layers.Activation('relu'),
        layers.Conv2D(64, (3, 3), padding='same'), layers.BatchNormalization(), layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)), layers.Dropout(0.25),
        
        layers.Conv2D(128, (3, 3), padding='same'), layers.BatchNormalization(), layers.Activation('relu'),
        layers.Conv2D(128, (3, 3), padding='same'), layers.BatchNormalization(), layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)), layers.Dropout(0.25),
        
        layers.Conv2D(256, (3, 3), padding='same'), layers.BatchNormalization(), layers.Activation('relu'),
        layers.Conv2D(256, (3, 3), padding='same'), layers.BatchNormalization(), layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)), layers.Dropout(0.25),
        
        layers.Conv2D(512, (3, 3), padding='same'), layers.BatchNormalization(), layers.Activation('relu'),
        layers.Conv2D(512, (3, 3), padding='same'), layers.BatchNormalization(), layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)), layers.Dropout(0.25),
        
        layers.Flatten(),
        layers.Dense(512, kernel_regularizer=regularizers.l2(0.001)), layers.BatchNormalization(), layers.Activation('relu'), layers.Dropout(0.4),
        layers.Dense(256, kernel_regularizer=regularizers.l2(0.001)), layers.BatchNormalization(), layers.Activation('relu'), layers.Dropout(0.4),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

model = build_cnn()
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1), metrics=['accuracy'])
model.summary()


In [ ]:
class SaveHistoryCallback(keras.callbacks.Callback):
    def __init__(self, history_path, checkpoint_path):
        super().__init__()
        self.history_path = history_path
        self.checkpoint_path = checkpoint_path
        self.history_data = {'accuracy': [], 'val_accuracy': [], 'loss': [], 'val_loss': [], 'lr': []}

    def on_epoch_end(self, epoch, logs=None):
        self.history_data['accuracy'].append(logs.get('accuracy'))
        self.history_data['val_accuracy'].append(logs.get('val_accuracy'))
        self.history_data['loss'].append(logs.get('loss'))
        self.history_data['val_loss'].append(logs.get('val_loss'))
        self.history_data['lr'].append(float(self.model.optimizer.learning_rate.numpy()))

        with open(self.history_path, 'wb') as f:
            pickle.dump(self.history_data, f)
        # Save checkpoint periodically or on best is handled by ModelCheckpoint, 
        # but we can save here too if needed. 
        # self.model.save(self.checkpoint_path)

callbacks = [
    ModelCheckpoint(filepath=BEST_MODEL_PATH, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-7, verbose=1),
    SaveHistoryCallback(HISTORY_PATH, MODEL_CHECKPOINT_PATH)
]
print("✅ Callbacks configured!")


In [ ]:
print("🚀 Starting Training...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)
print("✅ Training Completed!")


In [ ]:
# Load best model
best_model = keras.models.load_model(BEST_MODEL_PATH)

test_generator.reset()
test_loss, test_acc = best_model.evaluate(test_generator)
print(f"\n🎯 TEST ACCURACY: {test_acc*100:.2f}%")
print(f"   TEST LOSS: {test_loss:.4f}")

# Predictions
predictions = best_model.predict(test_generator, verbose=1)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

print(classification_report(y_true, y_pred, target_names=EMOTIONS, digits=4))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=EMOTIONS, yticklabels=EMOTIONS)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()
